In [11]:
from glob import glob
from PIL import Image
import imagehash

def frame(fname):
    """from a given file name, return the frame index.
    
    for example, 'cropped/001.png' returns int 1.
    """
    idx = int(fname.split("/")[1].split(".")[0])
    return idx

In [12]:
files = sorted(glob('cropped/*.png'))
files[:5]

['cropped/000.png',
 'cropped/001.png',
 'cropped/002.png',
 'cropped/003.png',
 'cropped/004.png']

In [13]:
stack = []

for i in range(len(files) - 1):
    prev_f = files[i]
    next_f = files[i + 1]
    
    prev_h = imagehash.average_hash(Image.open(prev_f))
    next_h = imagehash.average_hash(Image.open(next_f))

    if next_h == prev_h:
        stack.append(prev_f)
        stack.append(next_f)
    else:
        pass

In [14]:
stack[:5]

['cropped/001.png',
 'cropped/002.png',
 'cropped/004.png',
 'cropped/005.png',
 'cropped/005.png']

In [15]:
"""create continguous blocks of similar frames.
"""
MAX_FRAME_DIFF = 2

blocks = []
block = []

for i in range(len(stack) - 1):
    p_idx = frame(stack[i])
    n_idx = frame(stack[i + 1])
    
    if (n_idx - p_idx) < MAX_FRAME_DIFF:
        block.append(p_idx)
        block.append(n_idx)
    else:
        blocks.append(block)
        block = []

In [17]:
for block in blocks:
    block = set(block)
    if len(block) > 10:
        print(block)

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}
{34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48}
{54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84}
{270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283}
